In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import time
from skopt import gp_minimize
from skopt.space import Real, Integer
from tqdm import tqdm

: 

<h2 align="center"><strong>Implementación de un Modelo kMC para la Simulación de Procesos de Adsorción de Contaminantes Metálicos</strong></h2>



### **Creación del modelo**

In [42]:
class KineticMonteCarlo:
    def __init__(self, lattice_size, T, e_ads, e_diff, e_int_abs, time_array, time_factor):
        self.lattice_size = lattice_size
        self.lattice = np.zeros((lattice_size, lattice_size))
        self.T = T
        self.e_ads = e_ads
        self.e_diff = e_diff
        self.e_int_abs = e_int_abs
        self.k_b = 8.617e-5
        self.h = 4.136e-15
        self.k_0 = 5
        self.factor = time_factor
        self.time = time_array[0]
        self.time_array = time_array
        self.occupancy = {t: None for t in self.time_array}
        self.events_count = {"adsorption": 0, "diffusion": 0}
        self.adsorption = []
        self.diffusion = []

    def pick_random_site(self):
        i, j = np.random.randint(0, self.lattice_size, size=2)
        return i, j

    def count_occupied_neighbors(self, site):
        neighbors = 0
        i, j = site
        for ni, nj in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]:
            if 0 <= ni < self.lattice_size and 0 <= nj < self.lattice_size:
                neighbors += self.lattice[ni, nj]
        return neighbors

    def activation_energies(self, site):
        n_ij = self.count_occupied_neighbors(site)
        ads = self.e_int_abs * n_ij + self.e_ads
        return ads

    def calculate_transition_rates(self, site):
        E_ads= self.activation_energies(site)
        k_ads = (self.k_0 * self.k_b * self.T) / self.h * np.exp(-E_ads / (self.k_b * self.T))
        k_diff = (self.k_0 * self.k_b * self.T) / self.h * np.exp(-self.e_diff / (self.k_b * self.T))
        k_total = k_ads + k_diff
        return k_ads, k_diff, k_total

    def select_event(self, k_ads, k_diff):
        total_propensity = k_ads + k_diff
        r = np.random.rand() * total_propensity
        if r < k_ads:
            return "adsorption"
        else:
            return "diffusion"

    def run_simulation(self):
        end_time = self.time_array[-1]
        total_time = end_time - self.time_array[0]
        last_percentage = 0
        i=0
            

        while self.time < end_time:
            for t in self.time_array:
                if self.occupancy[t] is None and self.time >= t:
                    self.occupancy[t] = np.sum(self.lattice == 1) / (self.lattice_size ** 2) * 100

            site = self.pick_random_site()
            k_ads, k_diff, k_total = self.calculate_transition_rates(site)

            if k_total == 0:
                break  # No more events possible

            
            delta_t = (-np.log(np.random.rand()) / k_total)/self.factor
            self.time += delta_t
            event_type = self.select_event(k_ads, k_diff)

            if event_type == "adsorption":
                self.lattice[site] = 1
                self.events_count["adsorption"] += 1
            elif event_type == "diffusion":
                self.events_count["diffusion"] += 1
                if self.lattice[site] == 1:
                    neighbors = [
                        (site[0] + dx, site[1] + dy)
                        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
                        if 0 <= site[0] + dx < self.lattice_size and 0 <= site[1] + dy < self.lattice_size
                    ]
                    vacant_neighbors = [s for s in neighbors if self.lattice[s] == 0]
                    if vacant_neighbors:
                        new_site = vacant_neighbors[np.random.randint(len(vacant_neighbors))]
                        self.lattice[site], self.lattice[new_site] = 0, 1

            self.adsorption.append(self.events_count["adsorption"])
            self.diffusion.append(self.events_count["diffusion"])

        for t in self.time_array:
            if self.occupancy[t] is None and self.time >= t:
                self.occupancy[t] = np.sum(self.lattice == 1) / (self.lattice_size ** 2) * 100

        return self.occupancy

### **Datos experimentales**

In [ ]:
data = pd.read_csv('Datos recogidos.csv')
t = np.array([1,3,6,9,12,20,40,60])

for col in data.columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

data.dropna(inplace=True)
data.iloc[3] = data.iloc[3] * 10
data.reset_index(drop=True, inplace=True)
data.insert(0, 'T0', 0)


time_array = np.array([0, 1, 3, 6, 9, 12, 20, 40, 60])

plt.plot(time_array, data.iloc[5], marker='o', label='Experimental data')
plt.xlabel("Time")
plt.ylabel("Percentage of occupied sites")
plt.title("Adsorption over time")
plt.legend()
plt.show()

### **Compilación del modelo**

In [ ]:
lattice_size = 70
T = 300
e_ads = 0.09
e_diff = 0.294
e_int_abs = 0.1
time_array = np.array([0, 1, 3, 6, 9, 12, 20, 40, 60])
time_factor = 1e-7

KMC = KineticMonteCarlo(lattice_size=lattice_size, T=T, e_ads=e_ads, e_diff=e_diff, e_int_abs=e_int_abs,time_array=time_array, time_factor=time_factor)
print("Saturation: ",np.sum(KMC.lattice)/(lattice_size**2)*100, "%")

occupancy = KMC.run_simulation()
print(KMC.events_count)
print("Saturation: ",np.sum(KMC.lattice)/(lattice_size**2)*100, "%")
plt.imshow(KMC.lattice, cmap='binary')
plt.show()


In [ ]:
exp_data = data.iloc[8]
simulated_data = np.array(list(occupancy.values()))
simulated_data[0] = 0


plt.plot(time_array,exp_data,marker='o', label='Experimental data')
plt.plot(time_array,np.array(simulated_data), marker='o', label='Simulation data')
plt.legend()
plt.xlabel("Time")
plt.ylabel("Percentage of occupied sites")
plt.title("Adsorption over time")
plt.show()

mse = np.mean((np.array(exp_data) - np.array(simulated_data)) ** 2)
print(f"Mean Squared Error (MSE): {mse}")

### **Optimización Bayesiana**

In [46]:
base_params = {
    "T": 300,  
    "e_ads": 0.09,  
    "e_diff": 0.294,  
    "e_int_abs": 0.1
}
variability = 0.05

In [47]:
low_e_ads = min(base_params["e_ads"] * (1 - variability), base_params["e_ads"] * (1 + variability))
high_e_ads = max(base_params["e_ads"] * (1 - variability), base_params["e_ads"] * (1 + variability))

low_e_diff = min(base_params["e_diff"] * (1 - variability), base_params["e_diff"] * (1 + variability))
high_e_diff = max(base_params["e_diff"] * (1 - variability), base_params["e_diff"] * (1 + variability))

low_e_int_abs = min(base_params["e_int_abs"] * (1 - variability), base_params["e_int_abs"] * (1 + variability))
high_e_int_abs = max(base_params["e_int_abs"] * (1 - variability), base_params["e_int_abs"] * (1 + variability))

In [ ]:
# Definir número de iteraciones
N_CALLS = 200

# Inicializar barra de progreso
progress_bar = tqdm(total=N_CALLS, desc="Optimización Bayesiana en Progreso", position=0)

# Variables para almacenar el mejor resultado
best_params = None
best_error = float("inf")

def loss_function(params):
    global start_time, best_params, best_error
    
    # Desempaquetar parámetros
    e_ads, e_diff, e_int_abs, lattice_size, T = params  

    # Ejecutar el modelo KMC
    kmc_model = KineticMonteCarlo(
        lattice_size=lattice_size, T=T, e_ads=e_ads, 
        e_diff=e_diff, e_int_abs=e_int_abs, time_factor=1e-7,
        time_array=np.array([0, 1, 3, 6, 9, 12, 20, 40, 60])
    )
    kmc_model.run_simulation()

    # Calcular error
    simulated_data = np.array(list(kmc_model.occupancy.values()))
    error = np.mean((simulated_data - exp_data) ** 2)

    # Actualizar mejores parámetros si se encuentra un menor error
    if error < best_error:
        best_error = error
        best_params = params
        print(f"🟢 Nuevo mejor error: {best_error:.6f} con parámetros {best_params}")

    # Calcular tiempos estimados
    elapsed_time = time.time() - start_time
    avg_time_per_iter = elapsed_time / (progress_bar.n + 1)
    estimated_time_left = avg_time_per_iter * (N_CALLS - progress_bar.n - 1)

    # Agregar información en la barra de progreso
    progress_bar.set_postfix({
        "Tiempo transcurrido": f"{elapsed_time:.2f} s",
        "Tiempo restante": f"{estimated_time_left:.2f} s",
        "Mejor error": f"{best_error:.6f}"
    })
    
    progress_bar.update(1)

    return error

# Definir espacio de búsqueda
space = [
    Real(low_e_ads, high_e_ads, name="e_ads"),
    Real(low_e_diff, high_e_diff, name="e_diff"),
    Real(low_e_int_abs, high_e_int_abs, name="e_int_abs"),
    Integer(60, 100, name="lattice_size"),
    Real(280,350, name="T"),
]

# Iniciar optimización
start_time = time.time()
result = gp_minimize(loss_function, space, n_calls=N_CALLS, random_state=42)
progress_bar.close()

# Finalizar optimización
end_time = time.time()
print("\n✅ Optimización completada")
print("🔹 Parámetros óptimos encontrados:", best_params)
print("🔹 Error mínimo encontrado:", best_error)
print(f"⏳ Tiempo total de ejecución: {end_time - start_time:.2f} segundos")


In [ ]:
e_ads, e_diff, e_int_abs,lattice_size,T = best_params

kmc_model = KineticMonteCarlo(
    lattice_size= lattice_size, T=T, e_ads=e_ads, 
    e_diff=e_diff, e_int_abs=e_int_abs,time_factor=1e-7,
    time_array = np.array([0, 1, 3, 6, 9, 12, 20, 40, 60]), 
)
occupancy = kmc_model.run_simulation()
print(kmc_model.events_count)
print("Saturation: ",np.sum(kmc_model.lattice)/(lattice_size**2)*100, "%")
plt.imshow(kmc_model.lattice, cmap='binary')
plt.show()

In [ ]:
simulated_data = np.array(list(occupancy.values()))
simulated_data[0] = 0


plt.plot(time_array,exp_data,marker='o', label='Experimental data')
plt.plot(time_array,np.array(simulated_data), marker='o', label='Simulation data')
plt.legend()
plt.xlabel("Time")
plt.ylabel("Percentage of occupied sites")
plt.title("Adsorption over time")
plt.show()

mse = np.mean((np.array(exp_data) - np.array(simulated_data)) ** 2)
print(f"Mean Squared Error (MSE): {mse}")

### **Repetición hasta minimizar el error**
##### Como se trata de un modelo estocástico, diferentes simulaciones pueden generar diferentes resultados con pequeñas variaciones. Por lo que se compila varias veces el mismo modelo hasta minimizar el error con los datos experimentales.

In [ ]:
while mse >= 1.5:
    e_ads, e_diff, e_int_abs,lattice_size,T = best_params

    kmc_model = KineticMonteCarlo(
        lattice_size= lattice_size, T=T, e_ads=e_ads,
        e_diff=e_diff, e_int_abs=e_int_abs,time_factor=1e-7,
        time_array = np.array([0, 1, 3, 6, 9, 12, 20, 40, 60]), 
    )
    occupancy = kmc_model.run_simulation()

    simulated_data = np.array(list(occupancy.values()))
    simulated_data[0] = 0
    mse = np.mean((np.array(exp_data) - np.array(simulated_data)) ** 2)


print(kmc_model.events_count)
print("Saturation: ",np.sum(kmc_model.lattice)/(lattice_size**2)*100, "%")
plt.imshow(kmc_model.lattice, cmap='binary')
plt.show()

plt.plot(time_array,exp_data,marker='o', label='Experimental data')
plt.plot(time_array,np.array(simulated_data), marker='o', label='Simulation data')
plt.legend()
plt.xlabel("Time")
plt.ylabel("Percentage of occupied sites")
plt.title("Adsorption over time")
plt.show()

mse = np.mean((np.array(exp_data) - np.array(simulated_data)) ** 2)
print(f"Mean Squared Error (MSE): {mse}")

### **Relación Tamaño de Red vs Saturación**

In [ ]:
lattice_sizes = np.arange(60,200,10)
T = 300
e_ads = 0.09
e_diff = 0.294
e_int_abs = 0.1
time_array = np.array([0, 1, 3, 6, 9, 12, 20, 40, 60])
time_factor = 1e-7

saturations = []

for lattice_size in lattice_sizes:
    KMC = KineticMonteCarlo(lattice_size=lattice_size, T=T, e_ads=e_ads, e_diff=e_diff, 
                            e_int_abs=e_int_abs, time_array=time_array, time_factor=time_factor)
    occupancy = KMC.run_simulation()
    simulated_data = np.array(list(occupancy.values()))
    saturation = np.sum(KMC.lattice)/(lattice_size**2)*100
    saturations.append(saturation)

plt.plot(lattice_sizes, saturations, marker='o', linestyle='-', color='b')
plt.xlabel('Lattice Size')
plt.ylabel('Saturación (%)')
plt.title('Saturación vs Lattice Size')
plt.grid(True)
plt.show()

In [ ]:
dff=1